In [ ]:
import cartopy.crs as ccrs
import dask
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import scipy as sp
import torch
import torch_harmonics as th
import torch_harmonics.distributed as dist
import xarray

from postprocess_utils import *
from subs1_utils import precompute_latitudes

# Model Variables
In the following cell you can set the values of the variables relevant to the model. The details of each variable are included in the README. In most cases it is only necessary to set values for the standard variables. Note that any variable included in the model should be given the same value in the postprocess. For example, if the model used zw = 42 and kmax = 11, you should use zw = 42 and kmax = 11 below.

In [ ]:
# Set postprocess parameters.

# Standard Variables
zw = 63
expname = 'GammaT63L26_WarmPacOnly'
Topogdir = '/home/bkirtman/Work/Python/Scripts/AGCM/T63L26/InteractiveRainfall/'
dayst = 10800

# Advanced Variables
mw = None
imax = None
jmax = None
custom_path = None

In [ ]:
# Set Dependent Variables

# Check value for zw.
# Afterwards, set jmax and imax values based on the value given to zw.
# If a value is already given for one of the listed variables, use that instead
match zw:
    case 42:
        jmax = 64 if (jmax is None) else jmax
        imax = 128 if (imax is None) else imax
    case 63:
        jmax = 96 if (jmax is None) else jmax
        imax = 192 if (imax is None) else imax
    case 124:
        jmax = 188 if (jmax is None) else jmax
        imax = 376 if (imax is None) else imax
    case _:
        if (jmax is None) or (imax is None):
            raise Exception("Unexpected value for zw. Other values are implementable, but the user must specify values for jmax and imax in the advanced variables section.")

print("zw =", zw,
      "\njmax =", jmax,
      "\nimax =", imax,
      "\ndayst =", dayst)

In [ ]:
# Set datapath.
datapath = get_model_data_path(custom_path, expname)

# Set stamp for file names
stamp = 'days_1-' + str(dayst)
print("stamp =", stamp)

In [ ]:
#
fps = datapath+'lnps_1*.nc' 
dps = xarray.open_mfdataset(fps,decode_times=True,parallel = True)
#

In [ ]:
dps

In [ ]:
# Will need spectral transform topography to make sure we capture
# wiggles, so read in spectral topography from specific model simulation directory
# and convert to physical space

# Get the Gaussian latitudes and equally spaced longitudes
cost_lg, wlg, lats = precompute_latitudes(jmax)
lats = 90-180*lats/(np.pi)
lons = np.linspace(0.0,360.0-360.0/imax,imax)
Lon, Lat = np.meshgrid(lons, lats)
#
# Instantiate grid to spectral (dsht) and spectral to grid (disht) distibuted transforms
#
disht = dist.DistributedInverseRealSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
#
phismn = torch.load(Topogdir+'topog.spectral.pt')
phi = disht(phismn)
#

In [ ]:
#
# Now calculate sea level pressure data use lnps, topography & temperature data
# This is a very simple formula - probably better techniques available
#
ftemp = datapath+'temp_1*.nc'
dtemp = xarray.open_mfdataset(ftemp,decode_times=True, parallel=True)
#
surfp = (np.exp(dps.lnps))*1000.0
laps = -0.0065
grav = 9.8
rgas = 287.04
tstar = dtemp.t[:,25,:,:]*(1+laps*(rgas/grav)*(1-dtemp['lev'][25]))
yy = -phi.numpy()*laps/(rgas*tstar)
slp = surfp * np.exp(phi.numpy()/(rgas*tstar)*(1-yy/2+(yy*yy)/3)) #ECMWF Formula for Sea Level Pressure
#
#

In [ ]:
slp[500].plot()

In [ ]:
tstar[500].plot()

In [ ]:
times = slp['time']
lats = slp['lat']
lons = slp['lon']
dData = xarray.Dataset({'slp': (['time','lat','lon'],slp.values)},
                        coords={'time': times, 'lat': lats, 'lon': lons})
dData.to_netcdf(datapath+'sealevelpressure_'+stamp+'.nc')